<h1> Entrega final, ciencia de datos III </h1>

In [22]:
!pip install nltk
!pip install Textblob

In [36]:
#Se iomportan las bibliotecas. 

nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manue\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\manue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
#se carga el dataset
file_path=r'C:\Users\manue\OneDrive\Escritorio\SentimentAnalysis\data\new_test_data_s140.csv'
df=pd.read_csv(file_path, encoding='latin-1')

In [26]:
print("Datos originales")
print(df.head())

Datos originales
   Polarity  Id                          Date    Query          User  \
0         4   5  Mon May 11 03:18:54 UTC 2009  kindle2        chadfu   
1         4   6  Mon May 11 03:19:04 UTC 2009  kindle2         SIX15   
2         4   7  Mon May 11 03:21:41 UTC 2009  kindle2      yamarama   
3         4   8  Mon May 11 03:22:00 UTC 2009  kindle2  GeorgeVHulme   
4         0   9  Mon May 11 03:22:30 UTC 2009      aig       Seth937   

                                                Text  
0  Ok, first assesment of the #kindle2 ...it fuck...  
1  @kenburbary You'll love your Kindle2. I've had...  
2  @mikefish  Fair enough. But i have the Kindle2...  
3  @richardebaker no. it is too big. I'm quite ha...  
4  Fuck this economy. I hate aig and their non lo...  


In [27]:
#Renomrar las columnas para facilitar su uso
df=df.rename(columns={'0':'polarity', '1':'id', '2':'date', '3':'query', '4':'user', '5':'text'})


In [28]:
#Se limpia el texto
def clean_text(text):
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar menciones
    text = re.sub(r'@\w+', '', text)
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convertir a minúsculas
    text = text.lower()
    return text

In [29]:
# Aplicar la limpieza al texto
df['cleaned_text'] = df['Text'].apply(clean_text)

In [17]:
# 3. Eliminar stopwords

stop_words = set(stopwords.words('english')) 

df['cleaned_text'] = df['cleaned_text'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [18]:
# 4. Guardar el dataset preprocesado
output_file_path = r'C:\Users\manue\OneDrive\Escritorio\SentimentAnalysis\output\preprocessed_data.csv'
df.to_csv(output_file_path, index=False)

print("Preprocesamiento completado. Los datos preprocesados se guardaron en:", output_file_path)

Preprocesamiento completado. Los datos preprocesados se guardaron en: C:\Users\manue\OneDrive\Escritorio\SentimentAnalysis\output\preprocessed_data.csv


In [30]:
# Cargar el dataset preprocesado
file_path = r'C:\Users\manue\OneDrive\Escritorio\SentimentAnalysis\output\preprocessed_data.csv'
df = pd.read_csv(file_path)

In [31]:
# Tokenización
df['tokens'] = df['cleaned_text'].apply(word_tokenize)

In [32]:
# Función para obtener el análisis de sentimientos
def get_sentiment(text):
    analysis = TextBlob(text)
    # Retornar la polaridad
    return analysis.sentiment.polarity

In [33]:
# Aplicar el análisis de sentimientos
df['sentiment'] = df['cleaned_text'].apply(get_sentiment)

In [38]:
max_length = max([len(tokens) for tokens in df['tokens']])
vocab = set([word for tokens in df['tokens'] for word in tokens])
word_index = {word: i + 1 for i, word in enumerate(vocab)}  # +1 para reservar el índice 0
X = [[word_index[word] for word in tokens] for tokens in df['tokens']]
X = pad_sequences(X, maxlen=max_length, padding='post')

# Dividir los datos en entrenamiento y prueba
y = (df['sentiment'] > 0).astype(int)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [39]:
# 5. Construir la red neuronal
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\manue\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [40]:
# 6. Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.5955 - loss: 0.6838 - val_accuracy: 0.5300 - val_loss: 0.6804
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5943 - loss: 0.6642 - val_accuracy: 0.5300 - val_loss: 0.6774
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5884 - loss: 0.6566 - val_accuracy: 0.5400 - val_loss: 0.6741
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5710 - loss: 0.6573 - val_accuracy: 0.5500 - val_loss: 0.6699
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5780 - loss: 0.6416 - val_accuracy: 0.5600 - val_loss: 0.6645
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6158 - loss: 0.6277 - val_accuracy: 0.5900 - val_loss: 0.6581
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6847 - loss: 0.6034 - val_accuracy: 0.6000 - val_loss: 0.6526
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7168 - loss: 0.5814 - val_accuracy: 0.6000 - val_

In [41]:
# 7. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6685 - loss: 0.6191 
Accuracy: 0.6399999856948853


In [42]:
# 8. Uso del modelo para nuevos textos
new_text = "Este producto es increíble, lo recomiendo ampliamente."
new_tokens = word_tokenize(clean_text(new_text))
new_sequence = pad_sequences([[word_index.get(word, 0) for word in new_tokens]], maxlen=max_length)
sentiment = model.predict(new_sequence)[0][0]

if sentiment > 0.5:
    print("Predicción: Positivo")
else:
    print("Predicción: Negativo")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicción: Negativo
